In [ ]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
import functions as asf


In [ ]:
base_folder = './'  #end with /
os.chdir(base_folder)

darknet = "/home/peter/opt/darknet/"
model_folder = base_folder + 'trained_models/SSS_v5_128/'
obj_data = model_folder + 'obj.data'
cfg = model_folder + 'yolov4.cfg'
weights = model_folder + 'yolov4_best.weights'
thresh = 0.1


apply_folder = base_folder + 'apply/'
mosaic_folder = base_folder + 'apply/_todo/'
done_folder = base_folder + 'apply/_done/'
tile_folder = base_folder + "apply/tiles"

pixels = 128    # should correspond to model pixel size
greyscale = 1 #for SSS images yes, for multiband MBES image no
delete_background_images = 1
make_tfw = 0
convert_to_png = 1


In [ ]:
#Preparation
files = asf.getfiles('tif', mosaic_folder)
for file in tqdm(files):
    print("Working on file")
    full_path = mosaic_folder + file
    cmd = "mv " + full_path + " "   + apply_folder
    result = base_folder + file + str(pixels) + "detect_result.json"
    os.system(cmd)
    result_qgis = base_folder + file + str(pixels) + "detect_result.txt"

    #Empty tile folder
    for path, subdirs, files in os.walk(tile_folder):
        for filename in files:
            os.remove(path + '/' + filename)

    # convert to greyscale if needed
    if greyscale > 0:
        print("Conerting tp Greyscale")
        cmd = "python ./convert_to_greyscale.py " + apply_folder + " " + apply_folder + " --tag band1_ .tif "
        os.system(cmd)
        cmd = "mv " + apply_folder + file + " " + done_folder
        os.system(cmd)
        print("Moving ", file , " to Done Folder and continue with first band only")
        file = "band1_" + file

    # Cut into tiles
    print("Cutting")
    cmd = "python ./cut_image_to_tiles.py " + apply_folder + " " + tile_folder + " " +  str(pixels) +" .tif"
    os.system(cmd)

    # delete background images
    if delete_background_images > 0:
        cmd = "python ./delete_low_stddev_images.py " + tile_folder + " .tif 4.0 "
        os.system(cmd)

    if convert_to_png > 0:
        print("Convert to png")
        cmd = "python ./convert_to_png.py " + tile_folder
        os.system(cmd)

    if make_tfw > 0:
        print("Making tfw")
        cmd = "python ./create_tfw.py "+ tile_folder + " .tif 1 .tfw"
        os.system(cmd)

In [ ]:
#Make text file with files to be used
detect_path = apply_folder + "detect.txt"
#Make test and train.txt with link to files
with open(detect_path, "w") as a:
    for path, subdirs, files in os.walk(tile_folder):
        for filename in files:
            if filename.endswith(".png"):
                f = os.path.join(path, filename)
                a.write(str(f) + os.linesep)


In [ ]:
# Run detection and clean up
os.chdir(darknet)
cmd = "./darknet  detector test " +  obj_data + " " +  cfg  +" " + weights +" -dont_show -thresh " + str(thresh) + "  -ext_output -out "+ result +"  < "  + detect_path
os.system(cmd)

os.chdir(apply_folder)
cmd = "python ./../yolo_parse_results.py --directory=" + result + " --outfile="+result_qgis
os.system(cmd)

for path, subdirs, files in os.walk(tile_folder):
    for filename in files:
      os.remove(path + '/' + filename)

full_path = apply_folder + file
cmd = "mv " + full_path + " " + done_folder
os.system(cmd)